In [1]:
import google.generativeai as genai

model = genai.GenerativeModel('gemini-1.0-pro-latest')
response = model.generate_content("tell me a story")
print(response.text)

In the realm of Eldoria, where mystical forests whispered secrets and ancient ruins held forgotten lore, there lived a young adventurer named Anya. With her fiery red hair and a heart brimming with curiosity, she embarked on a perilous quest to uncover the lost city of Valoria.

Legends spoke of Valoria as a haven of knowledge and wonders, hidden deep within the treacherous Shadow Woods. Anya gathered a ragtag group of companions: Ethan, a skilled ranger with an uncanny knack for tracking, and Lyra, a wise mage whose spells could illuminate the darkest paths.

Together, they ventured into the sinister depths of the forest, where towering trees cast long shadows and the air hung heavy with ancient magic. As they pressed forward, Anya's keen eyes detected faint runes etched into the bark of an ancient oak. With Lyra's guidance, she deciphered their meaning, revealing a secret path leading to the lost city.

Through winding tunnels and crumbling walls, they navigated the ancient ruins of 

In [2]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a-b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[add, subtract, multiply, divide])


chat = model.start_chat(enable_automatic_function_calling=True)


response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text


'There are 2508 mittens in total.'

In [3]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)
     

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'There are 2508 mittens in total.'}]
--------------------------------------------------------------------------------


In [4]:
def find_movies(description: str, location: str = ''):
  """find movie titles currently playing in theaters based on any description, genre, title words, etc.

  Args:
      description: Any kind of description including category or genre, title words, attributes, etc.
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
  """
  return ['Barbie', 'Oppenheimer']

def find_theaters(location: str, movie: str = ''):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ['Googleplex 16', 'Android Theatre']

def get_showtimes(location:str, movie:str, theater:str, date:str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ['10:00', '11:00']


functions = {
    'find_movies': find_movies,
    'find_theaters': find_theaters,
    'get_showtimes': get_showtimes,
}

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=functions.values())
     

response = model.generate_content('Which theaters in Mountain View show the Barbie movie?')
response.candidates[0].content.parts

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}
]

In [5]:
def call_function(function_call, functions):
  function_name = function_call.name
  function_args = function_call.args
  return functions[function_name](**function_args)

part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
  result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']


In [6]:

import google.ai.generativelanguage as glm
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
s.update({'result': result})

# Update this after https://github.com/google/generative-ai-python/issues/243
function_response = glm.Part(
    function_response=glm.FunctionResponse(name='find_theaters', response=s))

# Build the message history
messages = [
    {'role':'user',
     'parts': ['Which theaters in Mountain View show the Barbie movie?.']},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': [function_response]}
]

# Generate the next response
response = model.generate_content(messages)
print(response.text)
     

OK. I found two theaters in Mountain View showing the Barbie movie: Googleplex 16 and Android Theatre.
